In [ ]:
import os, sys
import torch
from pathlib import Path
import numpy as np
import multiprocessing
USE_COLAB: bool = True
dataset_base_path = Path("/content/drive/My Drive/ECE 792 - Advance Topics in Machine Learning/Datasets/FakeFaces/PGGAN")
if USE_COLAB:
  from google.colab import drive
  
  # Mount the drive to access google shared docs
  drive.mount('/content/drive/', force_remount=True)

  if dataset_base_path.exists():
    print("Folder exists")
  else:
    print("DOESN'T EXIST. Add desired folder as a shortcut in your 'My Drive'")

Mounted at /content/drive/
Folder exists


In [ ]:
output_path = dataset_base_path
import shutil
if output_path.exists():
  shutil.rmtree(str(output_path))
print(output_path.exists())
output_path.mkdir(exist_ok=True, parents=True)
print(output_path.exists())
print(output_path)

False
True
/content/drive/My Drive/ECE 792 - Advance Topics in Machine Learning/Datasets/FakeFaces/PGGAN


In [ ]:
model = torch.hub.load('facebookresearch/pytorch_GAN_zoo:hub', 'PGAN', model_name='celebAHQ-512', pretrained=True, useGPU=True)
num_images = 40000
batch_size = 1
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torchvision
from torchvision.utils import save_image
from tqdm import tqdm

iterations = int(num_images / batch_size)
img_cnt = 0
for _ in tqdm(range(iterations)):
  noise, _ = model.buildNoiseData(batch_size)
  with torch.no_grad():
    generated_images = model.test(noise)

  image_size = 64
  transform = transforms.Compose([
      transforms.Resize(int(image_size * 1.1)),
      transforms.CenterCrop(image_size),
  ])
  generated_images = transform(generated_images)
  for img in generated_images:
    out_path = output_path / f"{img_cnt}.jpg"
    save_image(img, out_path)
    img_cnt += 1

Downloading: "https://github.com/facebookresearch/pytorch_GAN_zoo/zipball/hub" to /root/.cache/torch/hub/hub.zip
Downloading: "https://dl.fbaipublicfiles.com/gan_zoo/PGAN/celebaHQ16_december_s7_i96000-9c72988c.pth" to /root/.cache/torch/hub/checkpoints/celebaHQ16_december_s7_i96000-9c72988c.pth


  0%|          | 0.00/264M [00:00<?, ?B/s]

Average network found !


100%|██████████| 40000/40000 [09:05<00:00, 73.28it/s]


In [ ]:
# img_files = sorted(output_path.glob("*.jpg"))
# print(len(img_files))

output_zip_path = output_path.parent / "PGGAN.zip"
from zipfile import ZipFile
from tqdm import tqdm

# if output_zip_path.exists():
#   print(f"Deleting '{output_zip_path}'")
#   os.remove(str(output_zip_path))

# with ZipFile(str(output_zip_path), mode='w') as archive:
#   for file_ in tqdm(img_files):
#     archive.write(file_)

with ZipFile(str(output_zip_path), 'r') as zipObj:
  zipObj.extractall()

list_of_images_in_archive = zipObj.namelist()
len(list_of_images_in_archive)
# if len(list_of_images_in_archive) != len(img_files):
#   raise RuntimeError(f"NOT ALL IMAGES WERE SUCCESSFULLY ARCHIVED")

40000

In [ ]:
drive.flush_and_unmount()